In [2]:
import pandas as pd
import vectorbtpro as vbt
from datetime import datetime, timedelta
import pytz
import numpy as np
import pandas_ta as ta
import logging
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
import multiprocessing as mp
mp.set_start_method('fork')
from strategies import UnderwaterStrategy


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [3]:

start = "2018-01-01"
# Enter your parameters here
metric = 'total_return'

start_date = datetime(2020, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)

# The following is the number of days to look back for the analysis
time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
freq = '1h'

vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
# vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
# vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# binance_data.save("binance_data.pkl")

# If you already have the data downloaded, you can load it
# binance_data = vbt.BinanceData.load("binance_data.pkl")
# binance_data = binance_data.update() if you want to update it.
futures_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl'
# You can grab the necessary file from this google drive link. https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link

If you don't have the datafile yet you can grab it from this [link](https://drive.google.com/drive/folders/1jKy2DMbBow-J5jTvPutw-j17m7Ss8R3i?usp=drive_link)

In [4]:
btc = vbt.BinanceData.load(futures_path)
df = btc.get()

In [5]:
# create a signal array the same length as the dataframe
signal = np.zeros(len(df))
signal = pd.Series(signal, index=df.index)
# set the signal to 1 at midnight UTC
signal[df.at_time('00:00').index] = 1
# Create a -1 right before the signal
signal[df.at_time('12:00').index] = -1 # random short signals at noon
df['signal'] = signal
# Convert to int
df['signal'] = df['signal'].astype(int)
df['signal'].sum()

0

In [6]:
# This will be handy when we want to plot so it doesn't store all the html files in the root


def create_filename(stats_df):
    params = stats_df._strategy.__dict__['_params']
    filename = 'html_files/strat'
    # unpack the best params and add them to a filename
    for key, value in params.items():
        # keep the first 5 letters from each key
        key = key[:5]
        # just keep the first 4 digits from each value
        value= str(value)[:4]
        filename += f'_{key}_{value}_'
    filename += '.html'
    return filename

In [8]:
print(UnderwaterStrategy)

<class 'strategies.UnderwaterStrategy'>


In [ ]:

# run it on the full period
start = "2021-01-09"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2023-05-10"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    df.loc[start:end],
    UnderwaterStrategy,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1, # 0.5=2x leverage 1 = 1x leverage, .1 = 10x leverage
)
stats = bt.run() # Here we are using the best parameters from the optimization
filename = create_filename(stats)
print(stats)
bt.plot(resample='2h', filename=filename) # Note we are resampling to 2hours feel free to set it to FALSE to see how it really trades

In [8]:
filename = create_filename(stats)
print(filename)

html_files/.html


In [9]:
stats
print(stats)
bt.plot(resample=False, filename=filename)

Start                     2022-01-01 00:00...
End                       2023-06-03 23:12...
Duration                    518 days 23:12:00
Exposure Time [%]                   91.453648
Equity Final [$]                  111616681.5
Equity Peak [$]                   111841269.5
Return [%]                          11.616682
Buy & Hold Return [%]              -41.491126
Return (Ann.) [%]                    8.035556
Volatility (Ann.) [%]               23.318242
Sharpe Ratio                         0.344604
Sortino Ratio                        0.490093
Calmar Ratio                         0.388506
Max. Drawdown [%]                  -20.683231
Avg. Drawdown [%]                   -0.421241
Max. Drawdown Duration      406 days 05:49:00
Avg. Drawdown Duration        3 days 01:40:00
# Trades                                  898
Win Rate [%]                        23.385301
Best Trade [%]                      12.810546
Worst Trade [%]                    -43.835413
Avg. Trade [%]                    

GridPlot(id='p3118', ...)

In [10]:
bt.plot(resample=False)

GridPlot(id='p4515', ...)

## Now let's optimize the long short strategy
Note, this will take a while to run depending on how many parameters you want to optimize on.

I'm copying the class down here to remove the progress bars they conflict with the optimizer progress bars

In [11]:
stats, heatmap = bt.optimize(
    # initial_position_size = np.arange(0.3, 0.6, 0.1).tolist(),
    # percent_invested_threshold = 0.7 
    # atr_length = np.arange(7,37,10).to_list(), # 14 days
    atr_multiplier = np.arange(0.3, 1.5, 0.3).tolist(),
    # add_size = np.arange(0.10,0.30, 0.10).tolist(),
    # delay_period = np.arange(250,1500,500).tolist(),
    # delta_time = np.arange(1000,5000,1000).tolist(),
    upper_bound_profit_target = np.arange(0.03, 0.10, 0.02).tolist(),
    # lower_bound_loss_threshold = np.arange(-0.10, 0.03, 0.03).tolist(),
    take_profit_loss_reduction = np.arange(-0.15, -0.05, 0.05).tolist(), # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    # deleverage_pct = 0.5 # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    max_loss_threshold = np.arange(-0.05, -0.20, -0.05).tolist(),
    maximize='Equity Final [$]', 
    # maximize='Max. Drawdown [%]',# this can be any of the column names from the stats table the output of the backtest
    max_tries=200,
    random_state=0,
    return_heatmap=True)
best_params = stats._strategy.__dict__["_params"] # This will print out all the parameters used for the best backtest
print(f'Best Parameters: {best_params}')
heatmap.sort_values(ascending=False).iloc[:-5] # print the top 5 parameter sets

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Best Parameters: {'atr_multiplier': 0.6, 'upper_bound_profit_target': 0.03, 'take_profit_loss_reduction': -0.15, 'max_loss_threshold': -0.05}


atr_multiplier  upper_bound_profit_target  take_profit_loss_reduction  max_loss_threshold
0.6             0.03                       -0.15                       -0.10                 1.784164e+08
                                                                       -0.05                 1.784164e+08
                                                                       -0.20                 1.784164e+08
                                                                       -0.15                 1.784164e+08
                                           -0.10                       -0.20                 1.635696e+08
                                                                                                 ...     
                0.09                       -0.10                       -0.05                 1.062284e+08
                                                                       -0.20                 1.062284e+08
0.9             0.05                       -0.10              

In [16]:
print(stats)

Start                     2022-01-01 00:00...
End                       2023-06-03 23:12...
Duration                    518 days 23:12:00
Exposure Time [%]                   91.111221
Equity Final [$]                 178416435.73
Equity Peak [$]                  181186504.43
Return [%]                          78.416436
Buy & Hold Return [%]              -41.491126
Return (Ann.) [%]                   50.254672
Volatility (Ann.) [%]               35.460019
Sharpe Ratio                         1.417221
Sortino Ratio                        3.197443
Calmar Ratio                         1.950695
Max. Drawdown [%]                  -25.762448
Avg. Drawdown [%]                   -0.226024
Max. Drawdown Duration      146 days 18:47:00
Avg. Drawdown Duration        0 days 10:13:00
# Trades                                  377
Win Rate [%]                        54.907162
Best Trade [%]                      16.975912
Worst Trade [%]                    -41.732862
Avg. Trade [%]                    

In [12]:
# Plot the heatmap
from backtesting.lib import plot_heatmaps
plot_heatmaps(heatmap, agg='mean')


GridPlot(id='p5605', ...)

### Rerun the stats for the best strategy


In [13]:
bt.run(**best_params)
filename = create_filename(stats)
bt.plot(resample=False, filename=filename)

GridPlot(id='p6762', ...)

# Now go back and try optimizing with long only or short only

In [ ]:
# Just set the trade_type to LONG_ONLY and it will only take long trades EG. 
UnderwaterStrategy.trade_type = 'LONG_ONLY'

In [ ]:

# run it on the full period
start = "2021-01-09"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2023-05-10"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    df.loc[start:end],
    UnderwaterStrategy,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1, # 0.5=2x leverage 1 = 1x leverage, .1 = 10x leverage
)
stats = bt.run() # Here we are using the best parameters from the optimization
filename = create_filename(stats)
print(stats)
bt.plot(resample='2h', filename=filename) 